In [2]:
import torch
from transformers import LlamaForCausalLM, LlamaTokenizer
import transformers
MODEL_PATH = f"/shared/vsathia2/model/"

In [4]:
torch.cuda.is_available()

True

In [5]:
tokenizer = LlamaTokenizer.from_pretrained(MODEL_PATH, padding_side='left')

In [6]:
tokenizer.pad_token = tokenizer.eos_token

In [14]:
model = LlamaForCausalLM.from_pretrained(MODEL_PATH,device_map="cuda:0",attn_implementation="eager")

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [15]:
print(model)

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaRMSNorm()
  )
  (lm_head): Lin

In [144]:
# Place Hook on the MLP
glob_opt = []
glob_ipts = []
fp_i = 0
def echo_hook(module,input,output):
    global fp_i
    print(f"Forward Hook activated in MLP!")
    print(f"Input:{input}")
    print(f"Output: {output}")
    glob_opt.append(output.clone())
    glob_ipt.append(input)
    fp_i += 100
    return output

In [145]:
type(model.model.layers[0].mlp)

transformers.models.llama.modeling_llama.LlamaMLP

In [146]:
module_hook = model.model.layers[0].mlp

In [147]:
hook = module_hook.register_forward_hook(echo_hook)

In [148]:
hook

In [149]:
prompts=["test"]
inputs=tokenizer(prompts,return_tensors="pt",padding=True)

In [150]:
inputs

{'input_ids': tensor([[   1, 1243]]), 'attention_mask': tensor([[1, 1]])}

In [151]:
input_ids=inputs["input_ids"].cuda()

In [154]:
outputs=model(input_ids)
print(fp_i)

Forward Hook activated in MLP!
Input:(tensor([[[-0.0356, -0.0647, -0.0261,  ...,  0.0063, -0.0028,  0.0356],
         [-0.0343, -0.0434, -0.0423,  ...,  0.0757, -0.0577, -0.0238]]],
       device='cuda:0', grad_fn=<MulBackward0>),)
Output: tensor([[[-0.0052,  0.0948, -0.0104,  ..., -0.0180, -0.0250,  0.0413],
         [ 0.0241,  0.0151,  0.0100,  ...,  0.0152, -0.0059, -0.0149]]],
       device='cuda:0', grad_fn=<UnsafeViewBackward0>)
300


In [142]:
hook.remove()

In [125]:
glob_ipt

[(tensor([[[-0.0356, -0.0647, -0.0261,  ...,  0.0063, -0.0028,  0.0356],
           [-0.0343, -0.0434, -0.0423,  ...,  0.0757, -0.0577, -0.0238]]],
         device='cuda:0', grad_fn=<MulBackward0>),)]

In [76]:
outputs.logits.shape

torch.Size([1, 2, 32000])

In [131]:
hook.remove()

In [143]:
module_hook._forward_hooks

OrderedDict()

In [61]:
hook.remove()

In [69]:
from collections import OrderedDict
model.model.layers[0].mlp._forward_hooks = OrderedDict()

In [132]:
model.model.layers[0].mlp._forward_hooks

OrderedDict()

In [126]:
model.model.

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaRMSNorm()
  )
  (lm_head): Lin

In [128]:
model.config.hidden_size

4096